In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

In [ ]:
subset = pd.read_csv('SubsetBERT.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment")

tokenizedReviewsPackedIntoList = subset['content'].tolist()

# Convert tokenized input to tensors
inputs = tokenizer(tokenizedReviewsPackedIntoList, is_split_into_words=True,
                   return_tensors='pt', padding=True, truncation=True)

# Pass inputs through the model
with torch.no_grad():
    outputs = model(**inputs)

# Add 1 because the model outputs are zero-indexed
predictedScore = torch.argmax(outputs.logits, dim=1) + 1

# Extract confidence scores (highest softmax value for each prediction)
confidence = torch.softmax(
    outputs.logits, dim=1).max(dim=1).values * 100

subset['predictedScore'] = predictedScore.numpy()
subset['confidence'] = confidence.numpy()